In [ ]:
from datetime import datetime, timezone, timedelta
import os
import json
import requests
import base64
import wave

# --- Cấu hình ---
AI_API_BASE = "https://api.thucchien.ai/gemini/v1beta"
AI_API_KEY = "sk-YzYUqhbM9WskmD2AQDUXJQ"
MODEL = "gemini-2.5-flash-preview-tts:generateContent"
ENDPOINT = f"{AI_API_BASE}/models/{MODEL}"

# --- Cấu hình thư mục & log ---
AUDIO_DIR = "results_audio_multispeaker"
LOG_FILE_SUMMARY = "logs/audio_multispeaker_summary.log"
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs("logs", exist_ok=True)

# --- Setting đầu vào đa giọng nói ---
channels = 1
sample_width = 2  # 16-bit PCM = 2 bytes
sample_rate = 24000

# --- Nội dung đầu vào ---
text_input = """Giả sử hai người, Joe và Jane, đang trò chuyện với nhau. Joe mệt mỏi sau một ngày dài làm việc và muốn hỏi thăm Jane về ngày của cô ấy. Jane thì rất vui vẻ và hào hứng chia sẻ về những điều thú vị đã xảy ra trong ngày hôm nay.:
Speaker1: Xin chào Jane! Hôm nay bạn thế nào?
Speaker2: Chào Joe! Mình rất vui hôm nay. Còn bạn thì sao?"""

speaker_voice_configs = [
    {
        "speaker": "Joe",
        "voiceConfig": {"prebuiltVoiceConfig": {"voiceName": "Kore"}}
    },
    {
        "speaker": "Jane",
        "voiceConfig": {"prebuiltVoiceConfig": {"voiceName": "Puck"}}
    }
]

payload = {
    "contents": [{
        "parts": [{"text": text_input}]
    }],
    "generationConfig": {
        "responseModalities": ["AUDIO"],
        "speechConfig": {
            "multiSpeakerVoiceConfig": {
                "speakerVoiceConfigs": speaker_voice_configs
            }
        }
    }
}

headers = {
    "x-goog-api-key": AI_API_KEY,
    "Content-Type": "application/json"
}

# --- Thiết lập múi giờ Hà Nội ---
hanoi_tz = timezone(timedelta(hours=7))
timestamp = datetime.now(hanoi_tz).strftime("%Y-%m-%d %H:%M:%S")
timestamp_safe = datetime.now(hanoi_tz).strftime("%Y%m%d_%H%M%S")

# --- Gọi API ---
response = requests.post(ENDPOINT, headers=headers, json=payload)

if response.status_code != 200:
    print(f"❌ Lỗi API ({response.status_code}): {response.text}")
    exit(1)

# --- Phân tích JSON ---
try:
    result_json = response.json()
    # mime_type = result_json["candidates"][0]["content"]["parts"][0]["inlineData"]["mime_type"]
    audio_b64 = result_json["candidates"][0]["content"]["parts"][0]["inlineData"]["data"]
except Exception as e:
    print("❌ Không thể đọc dữ liệu âm thanh từ phản hồi API.")
    print(response.text)
    exit(1)
#
# # --- Giải mã base64 và lưu file ---
# if "rate=" in mime_type:
#     try:
#         rate_str = mime_type.split("rate=")[1].split(";")[0]
#         sample_rate = int(rate_str)
#     except:
#         sample_rate = 24000
# else:
#     sample_rate = 24000

audio_data = base64.b64decode(audio_b64)
audio_filename = f"audio_multispeaker_{timestamp_safe}.wav"  # Có thể đổi sang .mp3 nếu cần
save_path = os.path.join(AUDIO_DIR, audio_filename)

with wave.open(save_path, "wb") as f:
    f.setnchannels(channels)
    f.setframerate(sample_rate)
    f.setsampwidth(sample_width)
    f.writeframes(audio_data)

print(f"✅ File âm thanh đã lưu tại: {save_path}")

# --- Ghi log ---
log_entry = {
    "timestamp": timestamp,
    "model": MODEL,
    "input_text": text_input,
    "speakers": speaker_voice_configs,
    "saved_audio": save_path
}

with open(LOG_FILE_SUMMARY, "a", encoding="utf-8") as f:
    f.write(f"\n=================== 🕒 {timestamp} (UTC+7) ===================\n\n")
    f.write("🎙️ Input:\n")
    f.write(text_input + "\n\n")
    f.write("👥 Speakers:\n")
    for s in speaker_voice_configs:
        f.write(f"   - {s['speaker']}: {s['voiceConfig']['prebuiltVoiceConfig']['voiceName']}\n")
    f.write(f"\n📊 Model: {MODEL}\n")
    f.write(f"🎧 File âm thanh: {save_path}\n")
    f.write("\n======================================================================\n")

print(f"📝 Đã ghi log tóm tắt vào {LOG_FILE_SUMMARY}")
